## Collaborative Filtering - extract only PID, track_uri and pos 

In [1]:
import json
import numpy as np
from time import time
import pandas as pd
from pandas.io.json import json_normalize
import os
import concurrent.futures
import functools
import random
import gc
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
path = '../dataset/spotify_million_playlist_dataset/data/'

In [3]:
keys = [
    'pid',
#     'name',
#     'description',
#     'num_artists',
#     'num_albums',
#     'num_tracks',
#     'num_followers',
#     'duration_ms',
#     'collaborative',
    'tracks'
]

In [4]:
t0=time()
samp = 500
arr = np.empty((samp*300*1000,4), dtype = object)
filenames = os.listdir(path)
num = 0
for i, filename in enumerate(random.sample(sorted(filenames), samp)):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        D = pd.DataFrame(mpd_slice['playlists'])[keys]
        df = pd.DataFrame(D, columns=['pid','tracks'])
        idx = df.set_index(['pid']).tracks.apply(pd.Series).stack().index
        df = pd.DataFrame(df.set_index(['pid']).tracks.apply(pd.Series).stack().values.tolist(),index=idx).reset_index().drop('level_1',1)
        arr0 = df[['pid','pos','track_uri','artist_uri']].to_numpy()
        arr[num:num+len(df),:] = arr0
        num = num+len(df)
        
        print(filename,i)
# Time diff
print(f"Time taken: {(time()-t0)/60}")

mpd.slice.830000-830999.json 0
mpd.slice.591000-591999.json 1
mpd.slice.801000-801999.json 2
mpd.slice.734000-734999.json 3
mpd.slice.726000-726999.json 4
mpd.slice.154000-154999.json 5
mpd.slice.636000-636999.json 6
mpd.slice.661000-661999.json 7
mpd.slice.652000-652999.json 8
mpd.slice.564000-564999.json 9
mpd.slice.716000-716999.json 10
mpd.slice.403000-403999.json 11
mpd.slice.458000-458999.json 12
mpd.slice.791000-791999.json 13
mpd.slice.633000-633999.json 14
mpd.slice.226000-226999.json 15
mpd.slice.53000-53999.json 16
mpd.slice.218000-218999.json 17
mpd.slice.471000-471999.json 18
mpd.slice.883000-883999.json 19
mpd.slice.473000-473999.json 20
mpd.slice.934000-934999.json 21
mpd.slice.92000-92999.json 22
mpd.slice.284000-284999.json 23
mpd.slice.546000-546999.json 24
mpd.slice.349000-349999.json 25
mpd.slice.142000-142999.json 26
mpd.slice.109000-109999.json 27
mpd.slice.736000-736999.json 28
mpd.slice.64000-64999.json 29
mpd.slice.191000-191999.json 30
mpd.slice.696000-696999.

mpd.slice.45000-45999.json 254
mpd.slice.335000-335999.json 255
mpd.slice.562000-562999.json 256
mpd.slice.705000-705999.json 257
mpd.slice.933000-933999.json 258
mpd.slice.798000-798999.json 259
mpd.slice.727000-727999.json 260
mpd.slice.841000-841999.json 261
mpd.slice.998000-998999.json 262
mpd.slice.340000-340999.json 263
mpd.slice.357000-357999.json 264
mpd.slice.507000-507999.json 265
mpd.slice.345000-345999.json 266
mpd.slice.484000-484999.json 267
mpd.slice.385000-385999.json 268
mpd.slice.854000-854999.json 269
mpd.slice.147000-147999.json 270
mpd.slice.783000-783999.json 271
mpd.slice.390000-390999.json 272
mpd.slice.439000-439999.json 273
mpd.slice.690000-690999.json 274
mpd.slice.992000-992999.json 275
mpd.slice.863000-863999.json 276
mpd.slice.640000-640999.json 277
mpd.slice.807000-807999.json 278
mpd.slice.519000-519999.json 279
mpd.slice.90000-90999.json 280
mpd.slice.632000-632999.json 281
mpd.slice.187000-187999.json 282
mpd.slice.176000-176999.json 283
mpd.slice.8870

In [5]:
gc.collect()

20

In [6]:
arr.shape

(150000000, 4)

In [7]:
df = pd.DataFrame(arr)

In [8]:
df.columns=['pid','pos','track_uri', 'artist_uri']

In [9]:
df=df[df.isnull().any(axis=1)==False]

In [10]:
df.shape

(33211760, 4)

In [11]:
df.isnull().sum()

pid           0
pos           0
track_uri     0
artist_uri    0
dtype: int64

In [12]:
df.to_csv(f'../data-processed/scale-down/pid-track-artist-scaled-{samp}.csv', index = None)

## Add binary and scaled rating

In [13]:
df = pd.read_csv(f'../data-processed/scale-down/pid-track-artist-scaled-{samp}.csv')

In [14]:
df.head()

,pid,pos,track_uri,artist_uri
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D


In [15]:
df.dtypes

pid            int64
pos            int64
track_uri     object
artist_uri    object
dtype: object

### Binary scaling

In [16]:
df['binary_rating'] = 1

### Scale 1 to 10 

In [17]:
df.head()

,pid,pos,track_uri,artist_uri,binary_rating
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,1
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY,1
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D,1


In [18]:
df['pos_max'] = df.groupby('pid')['pos'].transform('max')
df['pos_rating'] = (9*(df['pos_max'] - df['pos'])/(df['pos_max']))+1
df=df.drop(columns=['pos_max'], axis =1 )

In [19]:
df.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,1,10.000
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY,1,9.875
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.750
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.625
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D,1,9.500


In [20]:
df[['pid','track_uri','binary_rating','pos_rating']].to_csv(f'../data-processed/pid-track-ratings-scaled-{samp}.csv', index = None)

## Hold out playlists for testing - to be used for R-precision and NDGC calculation

In [21]:
pids = pd.DataFrame(df.groupby('pid')['pos'].max()).reset_index()
pids.columns=['pid','num_tracks']
pids['train_test'] = np.where(np.random.rand(len(pids),1)<=0.7,'train','test')

In [22]:
pids.head()

,pid,num_tracks,train_test
0,0,51,train
1,1,38,test
2,2,63,test
3,3,125,train
4,4,16,test


In [23]:
D_num_tracks={}
for index, row in pids.iterrows():
    D_num_tracks[row['pid']] = row['num_tracks']

In [24]:
pids.describe()

,pid,num_tracks
count,500000.000000,500000.00000
mean,497895.500000,65.42352
std,291910.679732,53.71380
min,0.000000,4.00000
25%,237499.750000,25.00000
50%,509999.500000,48.00000
75%,752999.250000,91.00000
max,999999.000000,375.00000


In [25]:
holdout_array = [0.5,0.6,0.7,0.8,0.9,0.95]
first_random = ['first','random']

In [26]:
pids['test_first_random'] = np.random.randint(0, 2, pids.shape[0])

In [27]:
pids['test_hould_out_pct'] = [random.choice(holdout_array)*100 for i in range(len(pids))]
pids['seed_tracks_num'] = np.ceil(pids['num_tracks']*(1-pids['test_hould_out_pct']/100))
pids['seed_tracks_num'] = np.where(pids['seed_tracks_num']>100,100, pids['seed_tracks_num'])

In [28]:
pids['train_test'].value_counts()

train    350603
test     149397
Name: train_test, dtype: int64

In [29]:
D_test_train={}
for index, row in pids.iterrows():
    D_test_train[row['pid']] = row['train_test']

In [30]:
D_test_first_random={}
for index, row in pids.iterrows():
    D_test_first_random[row['pid']] = row['test_first_random']

In [31]:
D_seed_tracks_num={}
for index, row in pids.iterrows():
    D_seed_tracks_num[row['pid']] = row['seed_tracks_num']

In [32]:
df['train_test'] = df['pid'].map(D_test_train)
df['test_first_random'] = df['pid'].map(D_test_first_random)
df['seed_tracks_num'] = df['pid'].map(D_seed_tracks_num)

In [33]:
df['train_test'].value_counts()

train    23274032
test      9937728
Name: train_test, dtype: int64

In [34]:
df['num_tracks'] = df['pid'].map(D_num_tracks)

In [35]:
df.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating,train_test,test_first_random,seed_tracks_num,num_tracks
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,1,10.000,train,0,15.0,72
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY,1,9.875,train,0,15.0,72
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.750,train,0,15.0,72
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.625,train,0,15.0,72
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D,1,9.500,train,0,15.0,72


In [36]:
df_train = df[df.train_test == 'train']
df_test_1 = df[(df.train_test == 'test') & (df.test_first_random==1)]#first x tracks
df_test_2 = df[(df.train_test == 'test') & (df.test_first_random==0)]#random x tracks
df_train['hold_out'] = 0

In [37]:
df.shape[0]-df_test_1.shape[0]-df_test_2.shape[0]-df_train.shape[0]

0

In [38]:
df_test_1.shape, df_test_2.shape, df_train.shape

((4973975, 10), (4963753, 10), (23274032, 11))

In [39]:
df_train.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating,train_test,test_first_random,seed_tracks_num,num_tracks,hold_out
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,1,10.000,train,0,15.0,72,0
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY,1,9.875,train,0,15.0,72,0
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.750,train,0,15.0,72,0
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.625,train,0,15.0,72,0
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D,1,9.500,train,0,15.0,72,0


In [40]:
#select x first tracks
df_test_1['hold_out'] = np.where(df_test_1.pos<df_test_1.seed_tracks_num,0,1)

In [41]:
df_test_2['hold_out_'] = np.where(df_test_2.pos<df_test_2.seed_tracks_num,0,1)

In [42]:
df_test_1.shape

(4973975, 11)

In [43]:
df_test_2.shape

(4963753, 11)

In [44]:
D_seed_tracks={}
for index, row in pids.iterrows():
    D_seed_tracks[row['pid']] = row['seed_tracks_num']

In [45]:
D_num_rows={}
for index, row in pids.iterrows():
    D_num_rows[row['pid']] = row['num_tracks']+1

In [46]:
D_num_rows_sorted={}
for el in list(df_test_2.pid.unique()):
    D_num_rows_sorted[el] = D_num_rows[el]

In [47]:
D_seed_tracks_sorted={}
for el in list(df_test_2.pid.unique()):
    D_seed_tracks_sorted[el] = D_seed_tracks[el]

In [48]:
hold_out_list = []
for i,el in enumerate(list(df_test_2.pid.unique())):
    l=[0]*int(D_seed_tracks_sorted[el]) + [1]*(int(D_num_rows_sorted[el])-int(D_seed_tracks_sorted[el]))
    random.shuffle(l)
    hold_out_list.extend(l)

In [49]:
len(hold_out_list)

4963753

In [50]:
df_test_2['hold_out']=hold_out_list
df_test_2.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating,train_test,test_first_random,seed_tracks_num,num_tracks,hold_out_,hold_out
483,830008,0,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,spotify:artist:5M0lbkGluOPXLeFjApw8r8,1,10.000000,test,0,15.0,37,0,1
484,830008,1,spotify:track:6bqtKURdSWkInAJHDkuaL0,spotify:artist:3TVXtAsR1Inumwj472S9r4,1,9.756757,test,0,15.0,37,0,0
485,830008,2,spotify:track:7xVLFuuYdAvcTfcP3IG3dS,spotify:artist:5dHt1vcEm9qb8fCyLcB3HL,1,9.513514,test,0,15.0,37,0,1
486,830008,3,spotify:track:1eQBEelI2NCy7AUTerX0KS,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,1,9.270270,test,0,15.0,37,0,1
487,830008,4,spotify:track:4ImL3v98u2BLkwnyQDjfRm,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,1,9.027027,test,0,15.0,37,0,0


In [51]:
df_test_2.drop(columns=['hold_out_'], inplace=True)

In [52]:
df_test_2.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating,train_test,test_first_random,seed_tracks_num,num_tracks,hold_out
483,830008,0,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,spotify:artist:5M0lbkGluOPXLeFjApw8r8,1,10.000000,test,0,15.0,37,1
484,830008,1,spotify:track:6bqtKURdSWkInAJHDkuaL0,spotify:artist:3TVXtAsR1Inumwj472S9r4,1,9.756757,test,0,15.0,37,0
485,830008,2,spotify:track:7xVLFuuYdAvcTfcP3IG3dS,spotify:artist:5dHt1vcEm9qb8fCyLcB3HL,1,9.513514,test,0,15.0,37,1
486,830008,3,spotify:track:1eQBEelI2NCy7AUTerX0KS,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,1,9.270270,test,0,15.0,37,1
487,830008,4,spotify:track:4ImL3v98u2BLkwnyQDjfRm,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,1,9.027027,test,0,15.0,37,0


In [53]:
df2=df_train.append([df_test_1,df_test_2])

In [54]:
df2.shape, df.shape

((33211760, 11), (33211760, 10))

In [55]:
temp = df_test_1.groupby('pid')['hold_out','num_tracks'].agg({'hold_out':'sum','num_tracks':'max'})
temp['num_tracks']=temp['num_tracks']+1

<ipython-input-55-b6a8a00cedea>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = df_test_1.groupby('pid')['hold_out','num_tracks'].agg({'hold_out':'sum','num_tracks':'max'})


In [56]:
temp[temp['hold_out']>=temp['num_tracks']]

,hold_out,num_tracks
pid,,


In [57]:
temp[temp['hold_out']==0]

,hold_out,num_tracks
pid,,


In [58]:
df2.head()

,pid,pos,track_uri,artist_uri,binary_rating,pos_rating,train_test,test_first_random,seed_tracks_num,num_tracks,hold_out
0,830000,0,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,spotify:artist:0HjYETXAvcL6mzaKjAmH2K,1,10.000,train,0,15.0,72,0
1,830000,1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,spotify:artist:0jnsk9HBra6NMjO2oANoPY,1,9.875,train,0,15.0,72,0
2,830000,2,spotify:track:1BfPC2cHY3m9RIstE4cjr5,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.750,train,0,15.0,72,0
3,830000,3,spotify:track:0uwmgMXqoJ7wh102MTHTN7,spotify:artist:1YXg2Chm7UfZAUAkjJEpqK,1,9.625,train,0,15.0,72,0
4,830000,4,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,spotify:artist:7iZtZyCzp3LItcw1wtPI3D,1,9.500,train,0,15.0,72,0


In [59]:
df2.to_csv(f'../data-processed/scale-down/pid-track-ratings-train-test-tags-scaled-{samp}.csv', index = None)

In [61]:
df2[df2['hold_out'] == 0][['pid','track_uri','binary_rating']].to_csv(f'../data-processed/scale-down/pid-track-binary-rating-train-data-scaled-{samp}.csv', index = None)

In [62]:
df2[df2['hold_out'] == 0][['pid','track_uri','pos_rating']].to_csv(f'../data-processed/scale-down/pid-track-pos-rating-train-data-scaled-{samp}.csv', index = None)

In [63]:
evaluation_pids_ground_truth = df2[df2.train_test == 'test'][['pid','pos','track_uri','hold_out','test_first_random']]

In [64]:
evaluation_pids_ground_truth['seed_pattern'] = evaluation_pids_ground_truth['test_first_random'].map({1:'first n', 0:'random n'})

In [65]:
evaluation_pids_ground_truth[['pid','pos','track_uri','hold_out','seed_pattern']].to_csv(f'../data-processed/scale-down/evaluation-pids-ground-truth-scaled-{samp}.csv', index = None)

In [66]:
import pandas as pd
tracks_train_set=pd.read_csv(f'../data-processed/scale-down/pid-track-binary-rating-train-data-scaled-{samp}.csv')

In [67]:
tracks_train_set.head()

,pid,track_uri,binary_rating
0,830000,spotify:track:0oOCpb17DwBfvQ9Xbr9IYg,1
1,830000,spotify:track:6Knv6wdA0luoMUuuoYi2i1,1
2,830000,spotify:track:1BfPC2cHY3m9RIstE4cjr5,1
3,830000,spotify:track:0uwmgMXqoJ7wh102MTHTN7,1
4,830000,spotify:track:4jTiyLlOJVJj3mCr7yfPQD,1


In [68]:
pd.DataFrame(list(tracks_train_set.track_uri.unique()), columns=['track_uri']).shape

(1417389, 1)

In [69]:
pd.DataFrame(list(tracks_train_set.track_uri.unique()), columns=['track_uri']).to_csv(f'../data-processed/scale-down/track-uri-train-data-scaled.csv-{samp}', index = None)